In [35]:
import openai
import jieba
from transformers import (BertTokenizer, BertForMaskedLM, pipeline, Trainer, TrainingArguments,
                          GPT2LMHeadModel, GPT2Tokenizer, GPT2LMHeadModel, GPT2Tokenizer, TextDataset,
                          DataCollatorForLanguageModeling, Trainer, TrainingArguments)
from datasets import load_dataset, Dataset
import json
import torch
import torch.nn.functional as F
from torch.nn import Softmax
import pandas as pd
from collections import OrderedDict
import warnings
import numpy as np
from numpy.random import choice
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='master_sheet')
df = df[80:].reset_index(drop=True)
df

,Origin_condition,Index,Whole_S,Sframes,Completions,plaus_mean,plaus_std,exp_mean,exp_std,cloze(unexp_uncorrected),S_val_mean,S_val_std,S_arous_mean,S_arous_std,W_valence_mean,W_valence_std,W_arous_mean,W_arous_std,Use-Plaus(0:False/1:True),Use-S_valence(0:False/1:True)
0,Emo_SCU,0,有些人常怨天尤人自怨自艾，以博取他人關切,有些人常怨天尤人自怨自艾，以博取他人,關切,4.933333,1.569147,5.533333,1.203698,0.000000,3.357143,0.811273,3.142857,2.294625,5.166667,1.339447,3.333333,1.909727,1.0,0.0
1,Emo_SCE,0,有些人常怨天尤人自怨自艾，以博取他人同情,有些人常怨天尤人自怨自艾，以博取他人,同情,4.733333,1.806162,5.666667,1.398412,0.866667,3.357143,0.811273,3.142857,2.294625,5.437500,1.459166,3.375000,1.586401,1.0,0.0
2,Neu_SCU,1,阿公已無法自理生活，因此想要被看護關切,阿公已無法自理生活，因此想要被看護,關切,2.466667,1.586050,2.800000,1.514376,0.000000,3.714286,1.277753,2.714286,2.217739,5.166667,1.339447,3.333333,1.909727,0.0,1.0
3,Neu_SCE,1,阿公已無法自理生活，因此想要被看護照顧,阿公已無法自理生活，因此想要被看護,照顧,5.200000,1.720465,6.200000,0.832666,1.000000,3.714286,1.277753,2.714286,2.217739,5.500000,1.200490,2.944444,2.600277,1.0,1.0
4,Emo_WCE,2,男星在臉書放與辣妹親暱的合照，粉絲們說是女友,男星在臉書放與辣妹親暱的合照，粉絲們說是,女友,5.000000,1.712698,5.533333,1.499630,0.066667,4.285714,1.030158,3.142857,2.503059,5.937500,2.015564,3.750000,3.214550,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Neu_SCE,1405,小華明天要考英文，難怪他今天這麼認真在背單字,小華明天要考英文，難怪他今天這麼認真在背,單字,6.705882,0.455645,6.764706,0.424183,1.000000,5.000000,1.000000,1.928571,1.437472,4.933333,0.258199,1.866667,1.407463,1.0,1.0
483,Emo_SCE,1416,報告時教授不斷點頭稱讚，顯然對研究成果非常滿意,報告時教授不斷點頭稱讚，顯然對研究成果非常,滿意,6.764706,0.424183,6.764706,0.424183,0.950000,7.142857,1.406980,3.500000,2.584293,7.125000,1.087811,3.312500,1.851801,1.0,1.0
484,Emo_SCU,1416,報告時教授不斷點頭稱讚，顯然對研究成果非常賞識,報告時教授不斷點頭稱讚，顯然對研究成果非常,賞識,5.882353,1.277974,6.470588,0.499134,0.000000,7.142857,1.406980,3.500000,2.584293,6.875000,1.024695,3.562500,2.421260,1.0,1.0
485,Emo_SCU,1424,伴侶送我手寫的暖心卡片，是我收過最有意義的手作,伴侶送我手寫的暖心卡片，是我收過最有意義的,手作,4.470588,2.252258,5.823529,1.423967,0.000000,6.785714,1.697838,4.142857,2.799417,5.133333,0.351866,2.933333,2.016598,0.0,0.0


In [3]:
s_frames = df.iloc[[i for i in range(len(df)) if i%2==0]]['Sframes'].tolist()
clause = [] 
for i in s_frames:
    try:
        clause.append(i.split('，')[1])
    except IndexError:
        clause.append(i)

clause

['以博取他人',
 '因此想要被看護',
 '粉絲們說是',
 '放學時會在門口等',
 '很常看到',
 '最終成為了',
 '他竟不惜放棄家人的',
 '是想要展現出他的',
 '在困境中看見',
 '人與人相遇則是',
 '行徑又狡猾又',
 '設計得很',
 '別到父母都掛了才',
 '直到大家都能',
 '已經快要',
 '這裡的環境讓人覺得有些',
 '店家代表最終出面',
 '聽聽他的',
 '街友很意外但覺得好',
 '可以了解夫妻生活是否符合',
 '要跟愛慕的女孩',
 '我們已很久沒',
 '小明臉上的神情變得很',
 '在課堂上表現也很',
 '因此求職路上到處被公司',
 '在這裡小事情常被',
 '同志生活的',
 '主要是為了要讓學生瞭解這個',
 '便於今晨寫下',
 '還好後來有順利趕上',
 '原來是政府',
 '但大家都還沒',
 '憤怒的鄉民聚集凱道大聲',
 '看到他可以給他加油',
 '可以看見他們對未來人生的',
 '尊重子女的',
 '很感動收到男方婚宴過大禮送來的一盒',
 '到附近的店買了一塊',
 '當時我覺得我的生命好',
 '看起來很',
 '我想給女友一個',
 '買了筆和橡皮擦等開學要用的',
 '他常像洩了氣的皮球一樣神色',
 '難怪今天眼睛看起來有點',
 '搶救的希望',
 '地面一層厚厚的積雪也就會',
 '社會和經濟都有點',
 '是為了帶給觀眾內心一些',
 '原來是想送我',
 '我經常在辦公室分析各種財務',
 '在學習方面有非常棒的',
 '小瑜覺得其中最特別的就是',
 '因為可以收到聖誕老人的',
 '會和對方互相交換',
 '小雅當場放聲',
 '洗澡時常在浴室裡大聲',
 '要給女友大大的',
 '大家買了十二吋的',
 '連西施都會讚歎她的',
 '拍完片後要減肥以恢復戲前的',
 '頓失親人的家屬神情',
 '我會邀請大家一起',
 '妻子氣不過決定和他',
 '兩者可同時',
 '滿地都是',
 '看到山上有許多',
 '他們越來越',
 '這段關係終將',
 '只因為他成長的環境是',
 '每一口都讓人想到',
 '我內心充滿',
 '可以為內心帶來',
 '父母倆都哭到',
 '打掃完已經快要',
 '尚未找到殺了老教授的',
 '他終於找到和他合租公寓的',
 '

In [307]:
export = pd.DataFrame(data={'clause':clause})
export.to_excel('clauses.xlsx', index=False)

### Next token prediction with BERT

In [96]:
model_name = 'bert-base-chinese'
model = BertForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [217]:
raw = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences')
s = [x + raw['Optimal'][i] for i, x in enumerate(raw['Sframes'])]
df = pd.DataFrame(data={'sentences':s})
df

,sentences
0,有些人常怨天尤人自怨自艾，以博取他人同情
1,阿公已無法自理生活，因此想要被看護照顧
2,男星在臉書放與辣妹親暱的合照，粉絲們說是女友
3,每天上學都是爸爸接送的小明，放學時會在門口等爸爸
4,這險峻可怕的山路口號誌不明通行紊亂，很常看到車禍
...,...
970,這位工作人員做事十分認真，服務客人的品質很優良
971,他榮獲傑出教師獎，致力教學外也積極幫學生尋找出路
972,本書強力道盡患者的心聲，即使身上帶傷仍渴望被愛
973,大雄已慣於災難性思考，朋友都感到很無奈


In [91]:
key = 'my_key'
client = openai.OpenAI(api_key=key)
paraphrase = []
for i, sentence in enumerate(s):
    response = client.chat.completions.create(
    messages=[
            {
            'role': 'user',
            'content': '''請發揮創意根據這句話進行改寫、濃縮、或擴增, 
                          每句字數介於 20 至 26 之間 (請使用繁體中文): '''
                          + sentence,
            }
                 ],
            model='gpt-3.5-turbo',
        )

    paraphrase.append(response.choices[0].message.content)
paraphrase

['有人經常自憐自艾，向他人討同情，抱怨著所有事情都不如自己預期的那樣理想。',
 '阿公生活不能自理，希望有照顧者照顧。',
 '男星在臉書上和辣妹親密合照，粉絲們猜測可能是他的女友。',
 '每日上學由爸爸接送，放學後在門口等爸爸的小明，每天都希望看見他愛的父親。',
 '這山路險峻，交通口號誌不明，經常發生車禍，塞車。',
 '經過法律系畢業後，在律所實習，終於成為專業律師。',
 '為了私利，他竟不顧家人名譽放棄。',
 '這位演員說，他之所以參與拍片，就是為了呈現自己的魅力。',
 '人生總會遇到挫折，但困境中也會發現新的希望和機遇，就像柳暗花明。',
 '人與人相處需要真心，人與人相遇是命運。',
 '他時常設計陰險騙局引誘人們上當，行徑狡猾卑劣。',
 '這次的謎語真是千奇百怪、琳琅滿目，每個都令人驚豔，別有心思。',
 '浪費家產，冷漠孝道，勿待親人離世方悔悟，及時表現孝順與愛。',
 '老師總是反覆講解，直到眾人領悟。',
 '他因為哮喘發作，卻拼命熬夜工作，已瀕死。',
 '高山上突然降起大雨，猶如置身险境，環境讓人感到不安。',
 '這名店員態度十分惡劣，完全不符專業態度，店家最終代表出面致歉挽回形象。',
 '不妨詢問設計師，或許會有新的啟發。',
 '孩子們寒冷冬日送暖心禮物，街友驚訝感動。',
 '透過問卷調查，了解夫妻生活是否合乎預期。',
 '他買了鮮花和氣球，想向心儀的女孩表達愛意。',
 '朋友搬家後，就像失去了聯絡點，彷彿被時間吞沒，我們間隔擴大到無法逾越的距離。',
 '奶奶突然走了，小明臉上憂傷，眼神黯淡，心靈深處感到巨大的悲痛。',
 '小明從小就是個內向的孩子，課堂上的表現也很平凡。',
 '小凱因緊張而表現欠佳，令求職過程困難，連連遭公司拒絕。',
 '剛北上求學，習慣不同，小事變大事，讓我感到不適應，需要時間適應。',
 '這本書描述了同志在充滿仇恨與偏見年代中的艱辛生活，讓人深感震撼。',
 '今日課程旨在使學生了解特定概念，培養對知識深刻的理解。',
 '他痛苦到深夜，決意離世，今晨寫下絕筆遺書。',
 '上次出國行李安檢卡關，幸好最後趕上飛機。',
 '數百萬民眾渴望接種疫苗，政府卻未盡責未能如期供應，讓人感到失望。',
 '今年疫情突如其來，大家措手不及，各方尚未做好充分準備。',
 '憤怒鄉民因官員暴利欺壓

In [92]:
fine_tune_data = pd.DataFrame({'paraphrase':paraphrase})
fine_tune_data.to_excel('fine-tune-data.xlsx', index=False)

In [72]:
f = open('C:\\Users\\jowan\\Downloads\\data\\text-data.txt', 'r', encoding='utf-8')
s = f.read()
f.close()
s = s.translate({ord('\n'): None})
s = s.split('。')
s = [i + '。' for i in s]
df = pd.DataFrame({'sentences':s})
df = df.drop(9476)
df

,sentences
0,低潮沖刷時，我最怕走在路上，看見任何行人。
1,他們好像是成群結隊的異類生物。
2,我以宇宙間星球與星球的遙望距離，和他們疏遠。
3,罹患憂鬱症且獨居的我，正在隻身渡過生命的死蔭幽谷。
4,洩氣地坐在客廳沙發上。
...,...
9471,焚化爐前已經看不到大村女士的身影，偌大的頂樓只剩下我一個人。
9472,我慢慢地抬起頭，再次看著煙囪頂端。
9473,白煙斷斷續續，猶如細長的絲線。
9474,沒多久，最後一絲煙也隨風而逝。


In [97]:
dataset = Dataset.from_pandas(fine_tune_data)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'[PAD]'})
model.resize_token_embeddings(len(tokenizer))
dataset

Dataset({
    features: ['paraphrase'],
    num_rows: 975
})

In [98]:
def tokenize_function(examples):
    return tokenizer(examples['paraphrase'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/975 [00:00<?, ? examples/s]

Dataset({
    features: ['paraphrase', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 975
})

In [99]:
def mask_tokens(inputs, tokenizer):
    inputs = inputs.clone()
    labels = inputs.clone()
    probability_matrix = torch.full(labels.shape, 0.15)
    special_tokens_mask = [
        tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
    ]
    probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
    masked_indices = torch.bernoulli(probability_matrix).bool()
    labels[~masked_indices] = -100 # We only compute loss on masked tokens

    inputs[masked_indices] = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    return inputs, labels

In [100]:
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {"input_ids": mask_tokens(torch.tensor(examples["input_ids"]), tokenizer)[0],
                      "labels": mask_tokens(torch.tensor(examples["input_ids"]), tokenizer)[1]},
    batched=True
)

Map:   0%|          | 0/975 [00:00<?, ? examples/s]

In [101]:
training_args = TrainingArguments(
    output_dir='./models',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2
)

In [102]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=122, training_loss=0.2529622062307889, metrics={'train_runtime': 496.7376, 'train_samples_per_second': 1.963, 'train_steps_per_second': 0.246, 'total_flos': 64149138892800.0, 'train_loss': 0.2529622062307889, 'epoch': 1.0})

In [103]:
model.save_pretrained('fine-tuned-bert-mlm')
tokenizer.save_pretrained('fine-tuned-bert-mlm')

('fine-tuned-bert-mlm\\tokenizer_config.json',
 'fine-tuned-bert-mlm\\special_tokens_map.json',
 'fine-tuned-bert-mlm\\vocab.txt',
 'fine-tuned-bert-mlm\\added_tokens.json')

In [84]:
model.save_pretrained('fine-tuned-bert-mlm-2.0')
tokenizer.save_pretrained('fine-tuned-bert-mlm-2.0')

('fine-tuned-bert-mlm-2.0\\tokenizer_config.json',
 'fine-tuned-bert-mlm-2.0\\special_tokens_map.json',
 'fine-tuned-bert-mlm-2.0\\vocab.txt',
 'fine-tuned-bert-mlm-2.0\\added_tokens.json')

In [36]:
'''def get_all_predictions(masked_sentence):
    input_ids = tokenizer.encode(masked_sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits
    
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    mask_token_logits = predictions[0, mask_token_index, :].squeeze()
    probs = F.softmax(mask_token_logits, dim=-1)

    sorted_indices = torch.argsort(probabilities, descending=True)
    sorted_probs = probabilities[sorted_indices]

    tokens = tokenizer.convert_ids_to_tokens(sorted_indices.tolist())
    token_probs = sorted_probs.tolist()
    predictions = [{'score':p, 'token_str':t} for t,p in zip(tokens, token_probs)]

    return predictions'''

def select_token_based_on_prob(predictions):
    probs, tokens = [i['score'] for i in predictions], [i['token_str'] for i in predictions]
    prob_sum = sum(probs)
    probs = [i/prob_sum for i in probs]
    token = choice(tokens, 1, p=probs)[0]
    prob = probs[tokens.index(token)]
    return token, prob

def predict_last_word(sentence, token_len=1, language=None, model_name=None):
    if model_name == None:   
        if language == 'en':
            model_name = 'bert-base-uncased'
            mask, space, exe = ' [MASK].', ' ', True
        elif language == 'ch':
            model_name = 'bert-base-chinese'
            mask, space, exe = '[MASK]。', '', True
        else: 
            print('Languaged other than English and Chinese are not supported...')
            exe = False
            res_word, res_prob = None, None
    else:
        if language == 'en':
            mask, space, exe = ' [MASK].', ' ', True
        elif language == 'ch':
            mask, space, exe = '[MASK]。', '', True
        else: 
            print('Languaged other than English and Chinese are not supported...')
            exe = False
            res_word, res_prob = None, None  

    if exe == True:
        s, res_word, res_prob = (sentence + mask), '', 1.0
        for i in range(token_len):
            fill_mask = pipeline('fill-mask', model=model_name)
            predictions = fill_mask(s)
            #predictions = get_all_predictions(s)
            token, prob = predictions[0]['token_str'], predictions[0]['score']
            token, prob = select_token_based_on_prob(predictions)
            res_word += (token + space)
            res_prob *= prob
            sentence += space + token
            s = sentence + mask

    return res_word, res_prob

In [37]:
sentence = '重演這齣歷史的'
w, p = predict_last_word(sentence, token_len=2, language='ch')
print(w, p)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t

戲碼 0.6269996154966683


In [40]:
sentence = '重演這齣歷史的'
w, p = predict_last_word(sentence, token_len=2, language='ch', model_name='fine-tuned-bert-mlm')
print(w, p)

戲碼 0.46476585872992493


In [186]:
%%time

N = 20
clauses30 = pd.read_excel('clauses.xlsx')
results = [[] for s in clauses30['clause']]
for i, s in enumerate(clauses30['clause']):
    for j in range(N):
        w, p = predict_last_word(s, token_len=2, language='ch', model_name='fine-tuned-bert-mlm')
        results[i].append(w)
    results[i].insert(0, s)

cols = ['ans'+str(i+1) for i in range(N)]
cols.insert(0, 'sframe')
cloze = pd.DataFrame(index=[i for i in range(len(results))], columns=cols)
for i in range(len(results)):
    cloze.iloc[i] = results[i]
cloze

CPU times: total: 25min 55s
Wall time: 7min 16s


,sframe,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,...,ans11,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20
0,想要得到他人的,愛護,愛情,愛戴,愛情,愛情,心意,愛護,愛情,愛情,...,愛慕,愛情,愛情,愛情,爱情,愛情,好感,愛情,愛情,心意
1,我放學時會在公車站等,車子,他們,車去,車子,車子,他們,車輛,候他,車子,...,車去,他們,車子,他們,車輛,車子,車子,車去,你的,車子
2,他想要展現他的,美麗,愛情,美麗,美麗,愛情,愛情,美麗,美好,美好,...,美麗,心情,美貌,美貌,愛情,美麗,美好,美麗,愛情,美麗
3,在逆境中找到,爱情,你我,我们,我们,愛情,家园,你我,我们,我们,...,爱情,家园,愛情,我们,爱情,爱情,爱情,爱情,你我,家园
4,人與人之間是一種,情誼,情誼,愛情,緣分,情感,愛情,愛情,緣份,愛情,...,情愫,情緒,愛情,愛情,緣份,愛情,緣分,愛情,友情,情感
5,毆打女友的影星終於出面,了事,了情,了解,了情,了手,了解,了事,了事,了事,...,了事,了事,了解,了事,了解,了事,了解,了解,了情,了結
6,想看看當地的,人文,人民,人們,景觀,人民,魚群,人民,人民,人民,...,人民,魚群,景觀,人民,海鮮,人情,人多,人們,魚塘,人民
7,最近生活過得很,好過,好看,好過,爽朗,快活,好啊,好看,好看,好過,...,好過,好啊,好呢,快樂,好哦,好過,好啊,好看,好過,好看
8,他明天要跟心儀的對象,玩耍,聊天,聊天,說話,聊天,見面,談戀,見面,見面,...,談話,見面,玩耍,玩耍,談話,談心,聊天,說話,見面,說話
9,我們已經十年沒有,錢了,了了,錢賺,了家,變化,變化,了解,錢了,錢了,...,吃魚,變化,了了,了情,了家,玩過,了解,變化,了情,玩過


### GPT-3.5

In [26]:
def predict_last_word_gpt(sentence, model_name='gpt-3.5-turbo', temperature=1.0):
    prompt = '''請為以下缺少結尾詞的句子填入最適合的結尾詞，要求如下：
    1. 只輸出一個詞，不包含任何標點符號（如 “。”）
    2. 該詞必須是完整的繁體中文詞語，且適合作為句子的結尾\n   
    句子：'''
    
    response = client.chat.completions.create(
    messages=[
            {
            'role': 'user',
            'content': prompt + sentence
            }
                ],
            model=model_name,
    )

    yield response.choices[0].message.content

In [303]:
%%time

N = 20
cols = ['ans'+str(i+1) for i in range(N)]
cols.insert(0, 'sframe')

raw = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences')
sentences = [x for x in raw['Sframes']]
response = pd.DataFrame(index=[i for i in range(len(sentences))], columns=cols)
for i, s in enumerate(sentences[:100]):
    row = []
    for j in range(N):
        f = predict_last_word_gpt(s, model_name='gpt-3.5-turbo')
        row.append(next(f))
    row.insert(0, s)
    response.iloc[i] = row

response

CPU times: total: 5.72 s
Wall time: 17min 33s


,sframe,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,...,ans11,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20
0,有些人常怨天尤人自怨自艾，以博取他人,同情。,同情。,同情。,同情。,同情。,同情。,同情。,同情。,同情。,...,同情。,同情。,同情。,同情。,同情。,同情。,同情。,同情。,瞭解。,同情。
1,阿公已無法自理生活，因此想要被看護,照護。,照料。,帶走。.,服。,服務。,照顧。,。,親。,給。,...,照料。,人。,照料。,人照顧。,照顧。,希望。,人照顧。,照顧。,人。,縣。
2,男星在臉書放與辣妹親暱的合照，粉絲們說是,為炒作。,為了炒作。,為了營造話題性。,為煽情。,猜拳。,炫耀。,為炒作。,為了營造話題。,為炒作。,...,為了營造話題。,為了炒作。,為了炒作。,為炒作。,自導自演,為了炒作。,為了營造話題。,為了營造話題而做。,為了營銷。,為了宣傳。
3,每天上學都是爸爸接送的小明，放學時會在門口等,待。,爸爸。,著。,著。,著。,著。,著。,著。,爸爸。,...,著。,爸爸。,著。,車。,爸爸。,著。,爸爸。,著。,著。,候。
4,這險峻可怕的山路口號誌不明通行紊亂，很常看到,車禍。,車禍。,車禍。,違規情形。,車禍。,車禍。,車禍。,車禍。,車禍。,...,車禍。,車禍。,車禍。,車輛相撞。,車禍。,車禍。,車禍。,交通事故。,車禍。,車禍。
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
output = response.copy().head(100)
output.to_excel('responses.xlsx', index=False)

In [4]:
def rm_period(series):
    series2 = series.copy()
    for i, x in enumerate(series2):
        if '。' in x:
            series2[i] = x.replace('。', '')
    return series2

In [330]:
for i in range(len(output)):
    series = rm_period(output.iloc[i])
    output.iloc[i] = series

output

,sframe,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,...,ans11,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20
0,有些人常怨天尤人自怨自艾，以博取他人,同情,同情,同情,同情,同情,同情,同情,同情,同情,...,同情,同情,同情,同情,同情,同情,同情,同情,瞭解,同情
1,阿公已無法自理生活，因此想要被看護,照護,照料,帶走.,服,服務,照顧,,親,給,...,照料,人,照料,人照顧,照顧,希望,人照顧,照顧,人,縣
2,男星在臉書放與辣妹親暱的合照，粉絲們說是,為炒作,為了炒作,為了營造話題性,為煽情,猜拳,炫耀,為炒作,為了營造話題,為炒作,...,為了營造話題,為了炒作,為了炒作,為炒作,自導自演,為了炒作,為了營造話題,為了營造話題而做,為了營銷,為了宣傳
3,每天上學都是爸爸接送的小明，放學時會在門口等,待,爸爸,著,著,著,著,著,著,爸爸,...,著,爸爸,著,車,爸爸,著,爸爸,著,著,候
4,這險峻可怕的山路口號誌不明通行紊亂，很常看到,車禍,車禍,車禍,違規情形,車禍,車禍,車禍,車禍,車禍,...,車禍,車禍,車禍,車輛相撞,車禍,車禍,車禍,交通事故,車禍,車禍
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,檢察官調查後，明早會請證人出席,法庭,法庭,法庭,的,,法庭,庭,庭,法庭,...,法庭,庭,庭,庭,庭,法庭,法院,法庭,審訊,
96,小美和小光愛情長跑十年，最近他們決定,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,...,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚
97,因為現在家裡離公司太遠，於是最近我們決定要,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,...,搬家,搬家,搬家,租房,搬家,搬家,搬家,搬家,搬家,搬家
98,在國外錢包竟然被扒，警察搞了半天還是沒抓到,小偷,嫌犯,小偷,賊,小偷,賊,小偷,犯人,犯人,...,賊,罪犯,小偷,賊,賊,犯人,犯人,賊,小偷,賊


In [331]:
output.to_excel('responses-cleaned.xlsx', index=False)

In [333]:
df = pd.read_excel('responses-cleaned.xlsx')

In [5]:
def get_constraint(series, N=20):
    lst = series[1:].copy().to_list()
    w = max(lst,key=lst.count)
    count = lst.count(w)
    return count/N

constraint = [get_constraint(df.iloc[i]) for i in range(len(df))]
df.insert(21, 'constraint', constraint)
df

In [341]:
ratings = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences').head(100)
check = pd.DataFrame({'humans':ratings['Constraint'], 'gp-t3.5':df['constraint']})
check

,humans,gp-t3.5
0,0.866667,0.95
1,1.000000,0.15
2,0.066667,0.25
3,0.400000,0.55
4,0.812500,0.85
...,...,...
95,0.200000,0.40
96,1.000000,1.00
97,0.875000,0.95
98,0.400000,0.35


In [342]:
check.corr()

,humans,gp-t3.5
humans,1.000000,0.531113
gp-t3.5,0.531113,1.000000


In [15]:
raw = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences')[100:]
sframe = [x for i, x in enumerate(raw['Sframes'])]
completion = [x for x in raw['Optimal']]

to_jsonl = pd.DataFrame({'sframe':sframe, 'completion':completion})
to_jsonl

,sframe,completion
0,這位一線女演員憑藉這部劇爆紅，演技也收穫一致,好評
1,在山谷中大聲喊叫的話，可以在數秒後聽到自己聲音的,回音
2,受疫情影響許多人失業沒飯吃，因此急需政府趕緊,救助
3,剛出社會工作的兒子經濟尚未穩定，因此父母有幫忙,支援
4,和男友遠距離戀愛三年了，今天終於見面真是久別,重逢
...,...,...
870,這位工作人員做事十分認真，服務客人的品質很,優良
871,他榮獲傑出教師獎，致力教學外也積極幫學生尋找,出路
872,本書強力道盡患者的心聲，即使身上帶傷仍渴望被,愛
873,大雄已慣於災難性思考，朋友都感到很,無奈


In [16]:
with open('fine_tuning_data.jsonl', 'w', encoding='utf-8') as f:
    for i, row in to_jsonl.iterrows():
        record = {
                 'messages': [{'role':'user', 'content':row['sframe']},
                              {'role':'assistant', 'content':row['completion']}]        
                  }
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

In [17]:
key = 'my_key'
client = openai.OpenAI(api_key=key)

file_response = client.files.create(
    file=open('fine_tuning_data.jsonl', 'rb'),
    purpose='fine-tune'
)

file_id = file_response.id
print('File ID: ', file_id)

fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model='gpt-3.5-turbo' # As of now, GPT-3.5-turbo fine-tuning is not supported, use "davinci" or another model
)

print(fine_tune_response)

File ID:  file-MocWmTeseusOnIxpFqwjvl18
FineTuningJob(id='ftjob-J7PvA1HJjjQ3H690tp2qXLYk', created_at=1717489846, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-h3YFu8RXQqsBPWS08AF148PL', result_files=[], seed=1846731485, status='validating_files', trained_tokens=None, training_file='file-MocWmTeseusOnIxpFqwjvl18', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [19]:
fine_tune_job_id = fine_tune_response.id
response = client.fine_tuning.jobs.retrieve(fine_tune_job_id)
print(response)

FineTuningJob(id='ftjob-J7PvA1HJjjQ3H690tp2qXLYk', created_at=1717489846, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-h3YFu8RXQqsBPWS08AF148PL', result_files=[], seed=1846731485, status='running', trained_tokens=None, training_file='file-MocWmTeseusOnIxpFqwjvl18', validation_file=None, estimated_finish=1717494185, integrations=[], user_provided_suffix=None)


In [30]:
%%time

key = 'my_key'
client = openai.OpenAI(api_key=key)

N = 20
cols = ['ans'+str(i+1) for i in range(N)]
cols.insert(0, 'sframe')

raw = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences')
sentences = [x for x in raw['Sframes']]
response_finetuned = pd.DataFrame(index=[i for i in range(len(sentences))], columns=cols)
for i, s in enumerate(sentences[:100]):
    row = []
    for j in range(N):
        f = predict_last_word_gpt(s, model_name='ft:gpt-3.5-turbo-0125:personal::9WKz2PXU', temperature=1.5)
        row.append(next(f))
    row.insert(0, s)
    response_finetuned.iloc[i] = row

response_finetuned

CPU times: total: 6.34 s
Wall time: 17min 9s


,sframe,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,...,ans11,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20
0,有些人常怨天尤人自怨自艾，以博取他人,同情,同情,同情,同情,同情,同情,同情,同情,同情,...,同情,同情,同情,同情,同情,同情,同情,同情,同情,同情
1,阿公已無法自理生活，因此想要被看護,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,...,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧
2,男星在臉書放與辣妹親暱的合照，粉絲們說是,炒作,炒作,炒作,“炒作”,炒作,炒作,炒作,戲劇,炒作,...,炒作,炒作,炒作,炒作,炒作,炒作,炒作,炒作,炒作,炒作
3,每天上學都是爸爸接送的小明，放學時會在門口等,候著,候著,候著,候著,著,候著,候著,候著,候著,...,候著,著,爸爸,候著,著,候著,候著,著,著,候著
4,這險峻可怕的山路口號誌不明通行紊亂，很常看到,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,...,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
response_finetuned = response_finetuned.head(100)
constraint_finetuned = [get_constraint(response_finetuned.iloc[i]) for i in range(len(response_finetuned))]
response_finetuned.insert(21, 'constraint', constraint_finetuned)
response_finetuned

,sframe,ans1,ans2,ans3,ans4,ans5,ans6,ans7,ans8,ans9,...,ans12,ans13,ans14,ans15,ans16,ans17,ans18,ans19,ans20,constraint
0,有些人常怨天尤人自怨自艾，以博取他人,同情,同情,同情,同情,同情,同情,同情,同情,同情,...,同情,同情,同情,同情,同情,同情,同情,同情,同情,1.00
1,阿公已無法自理生活，因此想要被看護,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,...,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,照顧,1.00
2,男星在臉書放與辣妹親暱的合照，粉絲們說是,炒作,炒作,炒作,“炒作”,炒作,炒作,炒作,戲劇,炒作,...,炒作,炒作,炒作,炒作,炒作,炒作,炒作,炒作,炒作,0.90
3,每天上學都是爸爸接送的小明，放學時會在門口等,候著,候著,候著,候著,著,候著,候著,候著,候著,...,著,爸爸,候著,著,候著,候著,著,著,候著,0.65
4,這險峻可怕的山路口號誌不明通行紊亂，很常看到,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,...,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,車禍,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,檢察官調查後，明早會請證人出席,偵查,偵查,偵查,偵查,說明,審判,偵查,偵查,審判,...,調查,偵查,調查,說明,偵查,檢討,偵查,檢討,偵查,0.50
96,小美和小光愛情長跑十年，最近他們決定,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,...,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,結婚,1.00
97,因為現在家裡離公司太遠，於是最近我們決定要,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,...,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,搬家,1.00
98,在國外錢包竟然被扒，警察搞了半天還是沒抓到,小偷,小偷,小偷,賊,小偷,小偷,小偷,小偷,兇手,...,小偷,賊,小偷,小偷,賊,犯人,小偷,賊,小偷,0.60


In [32]:
ratings = pd.read_excel('master_sheet_v2(2023.07.18_1st_round).xlsx', sheet_name='All_sentences').head(100)
check_finetuned = pd.DataFrame({'humans':ratings['Constraint'], 'gp-t3.5':response_finetuned['constraint']})
check_finetuned

,humans,gp-t3.5
0,0.866667,1.00
1,1.000000,1.00
2,0.066667,0.90
3,0.400000,0.65
4,0.812500,1.00
...,...,...
95,0.200000,0.50
96,1.000000,1.00
97,0.875000,1.00
98,0.400000,0.60


In [33]:
check_finetuned.corr()

,humans,gp-t3.5
humans,1.000000,0.451103
gp-t3.5,0.451103,1.000000


In [41]:
lst = response_finetuned['constraint'].to_list()
print(lst)

[1.0, 1.0, 0.9, 0.65, 1.0, 1.0, 0.8, 0.6, 1.0, 1.0, 0.9, 1.0, 1.0, 0.9, 0.45, 0.55, 1.0, 0.65, 0.4, 0.75, 1.0, 1.0, 0.8, 0.55, 0.55, 1.0, 0.9, 0.85, 1.0, 1.0, 0.45, 0.9, 1.0, 0.85, 1.0, 0.95, 0.65, 1.0, 0.55, 1.0, 1.0, 1.0, 0.4, 0.95, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.95, 1.0, 1.0, 0.95, 1.0, 0.65, 1.0, 0.75, 0.9, 1.0, 0.9, 0.85, 0.85, 0.85, 1.0, 0.55, 0.6, 1.0, 1.0, 0.7, 0.95, 1.0, 1.0, 1.0, 0.7, 1.0, 0.95, 0.45, 1.0, 1.0, 1.0, 1.0, 0.65, 1.0, 1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.9, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 0.6, 0.7]
